In [44]:
import csv
from bpemb import BPEmb

In [2]:
pip install bpemb --user admin

  Using cached bpemb-0.3.2-py3-none-any.whl (19 kB)



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Using cached gensim-3.8.3-cp38-cp38-win_amd64.whl (24.2 MB)
  Using cached Cython-0.29.14-cp38-cp38-win_amd64.whl (1.7 MB)


In [97]:
import re
reg = re.compile("[!@#$%^&*,/\_-`~\n]")
sansList = []
with open('allData.csv', encoding = "utf-8") as csvfile:
    spamreader = csv.reader(csvfile)
    for i in spamreader:
        if i != "SANS":
            sansList.append(reg.sub(" ",i[1][7:][:-4]))

In [98]:
a = "श्रुत्वा हनुमतो वाक्यम् यथावद्भिभाषितम् |रामः प्रीतिसमायुक्तो वाक्यमुत्तरमब्रवीत् || ६-१-१"


In [99]:
sansList

['',
 'श्रुत्वा हनुमतो वाक्यम् यथावद्भिभाषितम् | रामः प्रीतिसमायुक्तो वाक्यमुत्तरमब्रवीत् || ६-१-१',
 'कृतम् हनुमता कार्यम् सुमहद्भुवि दुर्लभम् | मनसापि यदन्येन न शक्यम् धरणीतले || ६-१-२',
 'गच्छता मातुलकुलं भरतेन तदाऽनघः | शत्रुघ्नो नित्यशत्रुघ्नो नीतः प्रीतिपुरस्कृतः || २-१-१',
 'प्रविश्य तु महारण्यं दण्डकारण्यमात्मवान् | रामो ददर्श दुर्धर्षस्तापसाश्रममण्डलम् || ३-१-१',
 'न हि तम् परिपश्यामि यस्तरेत महोदधिम् | अन्यत्र गुरुडाद्वायोरन्यत्र च हनूमतः || ६-१-३',
 'स तत्र न्यवसद्भ्रात्रा सह सत्कारसत्कृतः | मातुलेनाश्वपतिना पुत्रस्नेहेन लालितः || २-१-२',
 'कुशचीरपरिक्षिप्तं ब्राह्म्यां लक्ष्यासमावृतम् | यथा प्रदीप्तं दुर्दर्शं गगने सूर्यमण्डलम् || ३-१-२',
 'देवदानवयक्षाणाम् गन्धर्वोरगरक्षसाम् | अप्रधृष्याम् पुरीम् लङ्काम् रावणेन सुरक्षिताम् || ६-१-४ प्रविष्टः सत्त्वमाश्रित्य जीवन्को नाम निष्क्रमेत् |',
 '',
 'तत्रापि निवसन्तौ तौ तर्प्यमाणौ च कामतः | भ्रातरौ स्मरतां वीरौ वृद्धं दशरथं नृपम् || २-१-३',
 'को विशेत्सुदुराधर्षाम् राक्षसैश्च सुरक्षिताम् || ६-१-५ यो वीर्यबलसम्पन्नो न समः स्याद्धनूम

In [46]:
bpemb_sa = BPEmb(lang="sa", vs=100000,dim=300)
bpemb_sa.encode(a)

downloading https://nlp.h-its.org/bpemb/sa/sa.wiki.bpe.vs100000.d300.w2v.bin.tar.gz


100%|████████████████████████████████████████████████████████████████| 112052493/112052493 [00:25<00:00, 4478679.68B/s]


['▁श्रुत्वा',
 '▁हनुम',
 'तो',
 '▁वाक्यम्',
 '▁यथाव',
 'द्भि',
 'भाष',
 'ितम्',
 '▁|',
 'रामः',
 '▁प्रीति',
 'सम',
 'ायुक्त',
 'ो',
 '▁वाक्य',
 'मुत्तर',
 'म',
 'ब्रवीत्',
 '▁|',
 '|',
 '▁0-0',
 '-0']

In [47]:
bpemb_sa = BPEmb(lang="sa", vs=100000)
bpemb_sa.encode(a)

['▁श्रुत्वा',
 '▁हनुम',
 'तो',
 '▁वाक्यम्',
 '▁यथाव',
 'द्भि',
 'भाष',
 'ितम्',
 '▁|',
 'रामः',
 '▁प्रीति',
 'सम',
 'ायुक्त',
 'ो',
 '▁वाक्य',
 'मुत्तर',
 'म',
 'ब्रवीत्',
 '▁|',
 '|',
 '▁0-0',
 '-0']

In [100]:
len(sansList)

18300

In [62]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(sansList)

In [68]:
len(matrix.toarray().sum(axis=0))

7099

In [101]:
c = [i for item in sansList for i in item.split()]

In [102]:
len(set(c))

88815

In [103]:
len(c)

291295

In [128]:
bpemb_sa = BPEmb(lang="sa", vs=100000,dim=300)
encodedSansList = []
for i in sansList:
    encodedSansList.append(bpemb_sa.encode(i))

In [129]:
encodedSansList[1:3]

[['▁श्रुत्वा',
  '▁हनुम',
  'तो',
  '▁वाक्यम्',
  '▁यथाव',
  'द्भि',
  'भाष',
  'ितम्',
  '▁|',
  '▁रामः',
  '▁प्रीति',
  'सम',
  'ायुक्त',
  'ो',
  '▁वाक्य',
  'मुत्तर',
  'म',
  'ब्रवीत्',
  '▁|',
  '|',
  '▁0-0',
  '-0'],
 ['▁कृतम्',
  '▁हनुमता',
  '▁कार्यम्',
  '▁सु',
  'मह',
  'द्भु',
  'वि',
  '▁दुर्लभम्',
  '▁|',
  '▁मनस',
  'ापि',
  '▁यद',
  'न्येन',
  '▁न',
  '▁शक्यम्',
  '▁धरण',
  'ीत',
  'ले',
  '▁|',
  '|',
  '▁0-0',
  '-0']]

In [119]:
rege = re.compile("[!@#$%^&*:,/\_`~\n]")
engList = []
with open('allData.csv', encoding = "utf-8") as csvfile:
    spamreader = csv.reader(csvfile)
    for i in spamreader:
        if i != "ENG":
            engList.append(rege.sub(" ",i[2][7:][:-4]))

In [120]:
engList[8]

'"Who can return alive  having entered (once) taking refuge only in self- command  the city of Lanka  which is unconquerable by gods demons and Yakshas and even by Gandharvas  Nagas and ogres and well under the custody of Ravana?"'

In [121]:
len(engList)

18300

In [122]:
e = [i for item in engList for i in item.split()]

In [123]:
len(e)

551924

In [124]:
len(set(e))

41351

In [125]:
bpemb_en = BPEmb(lang="en", vs=100000,dim=300)
encodedEngList = []
for i in engList:
    encodedEngList.append(bpemb_en.encode(i))

In [126]:
encodedEngList[1:10]

[['▁touched',
  '▁with',
  '▁joy',
  '▁to',
  '▁hear',
  '▁the',
  '▁words',
  '▁rightly',
  '▁spoken',
  '▁by',
  '▁hanuman',
  '▁rama',
  '▁replied',
  '▁as',
  '▁follows',
  '▁-'],
 ['▁"',
  'a',
  '▁very',
  '▁outstanding',
  '▁work',
  '▁the',
  '▁most',
  '▁arduous',
  '▁in',
  '▁the',
  '▁world',
  '▁has',
  '▁been',
  '▁done',
  '▁by',
  '▁hanuman',
  '▁which',
  '▁could',
  '▁not',
  '▁be',
  '▁carried',
  '▁out',
  '▁even',
  '▁in',
  '▁thought',
  '▁by',
  '▁any',
  '▁other',
  '▁on',
  '▁the',
  '▁surface',
  '▁of',
  '▁this',
  '▁earth',
  '."'],
 ['▁bhar',
  'atha',
  '▁while',
  '▁going',
  '▁to',
  '▁his',
  '▁maternal',
  '▁uncle',
  "'",
  's',
  '▁house',
  '▁has',
  '▁taken',
  '▁his',
  '▁brother',
  '▁sat',
  'r',
  'ugh',
  'na',
  '▁(',
  'he',
  '▁who',
  '▁has',
  '▁no',
  '▁sins',
  '▁and',
  '▁who',
  '▁annih',
  'il',
  'ates',
  '▁his',
  '▁enemies',
  ')',
  '▁along',
  '▁with',
  '▁him',
  '▁with',
  '▁love',
  '.'],
 ['▁on',
  '▁entering',
  '▁the',
  '

In [127]:
len(encodedEngList)

18300